In [63]:
import pandas as pd
import sqlalchemy as sqla
import numpy as np

In [9]:
##Setting Up SQL Engine and Extracting Data
engine = sqla.create_engine('sqlite:///top200data')
pd.read_csv('top200data.csv').to_sql('top200data', engine)

In [1]:
##Testing Spotipy API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [40]:
##Test Extractiong of Genres 
results = spotify.search(q='artist:' + 'Los Pikantes', type='artist')
results['artists']['items'][0]['genres']
# items = results['artists']['items']
# items['items']

[]

In [39]:
##Extracting Unique List of Artists
artistlist = pd.DataFrame(columns=['artist','genre'])

##Only Counting Main Artist for Collaborations
artistlist['artist'] = pd.read_sql('SELECT DISTINCT artist FROM top200data WHERE artist IS NOT NULL',engine)['artist'].apply(lambda x: x.split(', ')[0]).unique()
artistlist
for i in range(len(artistlist)):
    try:
        results = spotify.search(q='artist:' + artistlist.loc[i,'artist'], type='artist')
        artistlist.loc[i,'genre']=results['artists']['items'][0]['genres']
    except:
        pass
artistlist

,artist,genre
0,Ed Sheeran,"[pop, uk pop]"
1,Migos,"[atl hip hop, hip hop, pop rap, rap, southern ..."
2,Drake,"[canadian hip hop, canadian pop, hip hop, pop ..."
3,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]"
4,ZAYN,"[dance pop, pop, post-teen pop, uk pop]"
...,...,...
2515,Los Rehenes,"[grupera, gruperas inmortales, regional mexica..."
2516,Sonido Caracol,"[cumbia pop, cumbia uruguaya, plena uruguaya]"
2517,Karibe con K,[cumbia uruguaya]
2518,Chacho Ramos,"[cumbia pop, cumbia uruguaya, plena uruguaya]"


In [83]:
##Assgining NaNs for Artists with No Genres Associatied
artistlist.loc[(artistlist['genre'].str.len() == 0),"genre"]=np.nan

In [85]:
##Export Data
artistlist.to_csv("artistsdata.csv")